<a href="https://colab.research.google.com/github/athu-lya/Assignment1/blob/main/Internship_Project_Credit_Score_Classification_Model_Development_Athulya_KM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#checking Enviornment
print('hello world')

In [ ]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

## Build a Classification model with Highest Accuracy, Precison and Recall score. Create a predictive model that categorizes customers into three creditworthiness levels: Good, Standard, and Poor with the given data.

In [ ]:
#loading the dataset to the enviornment
credit=pd.read_csv('/content/credit (1).csv')

In [ ]:
credit.head()

#EDA(Exploratory Data Analysis)

In [ ]:
credit.shape

there are 100000 rows and 28 columns in the dataset

In [ ]:
credit.info()

Dataset has 28 features and 100000 entries. Among that, 21 features are numerical (18 float and 3 integer) and 7 are categorical features.

In [ ]:
credit.describe()

In [ ]:
credit.dtypes

In [ ]:
credit.columns

In [ ]:
#check for duplicates
credit.duplicated().sum()

there is no duplicates in the dataset

In [ ]:
#chech for unique values
(credit.nunique()).sort_values(ascending=False)

Feature 'ID' and ''Credit_Utilization_Ratio' have full of unique values

In [ ]:
credit.drop(['ID','Customer_ID','Month','Name','SSN','Occupation','Type_of_Loan','Credit_Utilization_Ratio'],axis=1,inplace=True)

droped some unnecessary features by assuming that they don’t have any predictive power to predict the target variable.

In [ ]:
#check for null values
credit.isnull().sum()

There is no null values in the dataset

In [ ]:
#Visualization
plt.figure(figsize=(10, 6))
sns.histplot(credit['Age'], kde=True, bins=30)
plt.title("Histogram of Age")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

In [ ]:
sns.countplot(x = credit['Credit_Mix'],palette = "colorblind");
plt.xticks(rotation = 90)

In [ ]:
credit["Credit_Score"].value_counts().plot.pie(explode = [0.03,0.03,0.03], autopct="%1.2f%%",shadow = True,labels = ["Standard","Poor","Good"])
plt.title('Distribution of Credit_Score')
plt.show()

Data is imbalanced since the majority lies in 'standard' i.e, 53.17%, 29% in 'poor' and less in 'good'(17.83%)

In [ ]:
#Correlation
plt.figure(figsize=(16, 8))
sns.heatmap(credit.select_dtypes(include="number").corr(),annot=True)
plt.show()

'Annual_Income' and 'Monthly_Inhand_Salary' have correlation 1

#**Outlier detection and treatment**

In [ ]:
#check for outliers
plt.figure(figsize=(16, 8))
sns.boxplot(credit)
plt.xticks(rotation=90)
plt.show()

there are outliers present in the dataset

In [ ]:
new_columns=['Annual_Income','Monthly_Inhand_Salary','Delay_from_due_date', 'Changed_Credit_Limit','Num_Credit_Inquiries', 'Outstanding_Debt', 'Total_EMI_per_month','Amount_invested_monthly', 'Monthly_Balance']

In [ ]:
for i in new_columns:
    print('Skewness of feature',i,':',credit[i].skew())

All 9 features having outliers are skewed distributions. So using IQR (Inter Quartile Range) method for outlier detection and clip function for outlier treatment.

In [ ]:
for i in new_columns:
  plt.figure(i)
  plt.boxplot(credit[i])
  plt.title(i);

In [ ]:
for j in ['Annual_Income','Monthly_Inhand_Salary','Delay_from_due_date', 'Changed_Credit_Limit','Num_Credit_Inquiries', 'Outstanding_Debt', 'Total_EMI_per_month','Amount_invested_monthly', 'Monthly_Balance']:
    Q1=np.percentile(credit[j],25,method='midpoint')
    Q2=np.percentile(credit[j],50,method='midpoint')
    Q3=np.percentile(credit[j],75,method='midpoint')
    IQR = Q3-Q1
    low_lim = Q1-1.5*IQR
    up_lim = Q3+1.5*IQR
    outliers = []
    for h in credit[j]:
        if ((h < low_lim) | (h > up_lim)):
            outliers.append(h)
    credit[j]=credit[j].clip(lower=low_lim,upper=up_lim)

In [ ]:
new_columns1=['Annual_Income','Monthly_Inhand_Salary','Delay_from_due_date', 'Changed_Credit_Limit','Num_Credit_Inquiries', 'Outstanding_Debt', 'Total_EMI_per_month','Amount_invested_monthly', 'Monthly_Balance']

In [ ]:
plt.figure(figsize=(16, 8))
sns.boxplot(credit)
plt.xticks(rotation=90)
plt.show()

All outliers are treated properly. The remaining outliers are the outliers of newly transformed data.

In [ ]:
credit.columns

In [ ]:
#Label encoding
le = LabelEncoder()
cat_columns=['Credit_Mix','Credit_History_Age','Payment_of_Min_Amount','Credit_Score']
for i in cat_columns:
    credit[i]=le.fit_transform(credit[i])

In [ ]:
credit.info()

In [ ]:
#ordinal encoding
credit['Payment_Behaviour'].unique()

In [ ]:
Payment_Behaviour_cat=['Low_spent_Small_value_payments','Low_spent_Medium_value_payments','Low_spent_Large_value_payments',
                          'High_spent_Small_value_payments','High_spent_Medium_value_payments','High_spent_Large_value_payments']
Payment_Behaviour_encoder=OrdinalEncoder(categories=[Payment_Behaviour_cat])
credit['Payment_Behaviour']=Payment_Behaviour_encoder.fit_transform(credit[['Payment_Behaviour']])


In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(credit.corr(),annot=True)
plt.show()

Since the correlation value of 'Annual_Income' and 'Monthly_Inhand_Salary' are one, they are highly correlated. so that droping 'Monthly_Inhand_Salary'.

In [ ]:
credit.drop('Monthly_Inhand_Salary',axis=1,inplace=True)

In [ ]:
credit.head()

In [ ]:
#setting the target variable and independentb variables
x=credit.drop('Credit_Score',axis=1)
y=credit['Credit_Score']
y=pd.DataFrame(y,columns=['Credit_Score'])

In [ ]:
sc=StandardScaler()
x_sc=sc.fit_transform(x)
x=pd.DataFrame(x_sc,columns=x.columns)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [ ]:
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)
x_val=sc.fit_transform(x_val)
x_test=sc.fit_transform(x_test)

#Modelling

Decision Trees

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
decision_tree_pred = decision_tree.predict(x_test)
decision_tree_accuracy = accuracy_score(y_test, decision_tree_pred)
decision_tree_precision = precision_score(y_test, decision_tree_pred, average='macro')
decision_tree_recall = recall_score(y_test, decision_tree_pred, average='macro')
decision_tree_f1 = f1_score(y_test, decision_tree_pred, average='macro')
print("Decision Tree Accuracy:", decision_tree_accuracy)
print("confusion_matrix:", confusion_matrix(y_test,decision_tree_pred))
print("precision_score:",decision_tree_precision)
print("recall_score:",decision_tree_recall)
print("f1_score:",decision_tree_f1)

Random Forest

In [ ]:
random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)
random_forest_pred = random_forest.predict(x_test)
random_forest_accuracy = accuracy_score(y_test, random_forest_pred)
random_forest_precision = precision_score(y_test, random_forest_pred, average='macro')
random_forest_recall = recall_score(y_test, random_forest_pred, average='macro')
random_forest_f1 = f1_score(y_test, random_forest_pred, average='macro')
print("Random Forest Accuracy:", random_forest_accuracy)
print("confusion_matrix:", confusion_matrix(y_test,random_forest_pred))
print("precision_score:",random_forest_precision)
print("recall_score:",random_forest_recall)
print("f1_score:",random_forest_f1)

Logistic Regression

In [ ]:
logistic_reg = LogisticRegression()
logistic_reg.fit(x_train, y_train)
logistic_reg_pred = logistic_reg.predict(x_test)
logistic_reg_accuracy = accuracy_score(y_test, logistic_reg_pred)
logistic_precision = precision_score(y_test, logistic_reg_pred, average='macro')
logistic_recall = recall_score(y_test, logistic_reg_pred, average='macro')
logistic_f1 = f1_score(y_test, logistic_reg_pred, average='macro')
print("logistic_reg_accuracy:", logistic_reg_accuracy)
print("confusion_matrix:", confusion_matrix(y_test,logistic_reg_pred))
print("precision_score:",logistic_precision)
print("recall_score:",logistic_recall)
print("f1_score:",logistic_f1)

SVM

In [ ]:
svm = SVC()
svm.fit(x_train, y_train)
svm_pred = svm.predict(x_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred, average='macro')
svm_recall = recall_score(y_test, svm_pred, average='macro')
svm_f1 = f1_score(y_test, svm_pred, average='macro')
print("SVM Accuracy:", svm_accuracy)
print("confusion_matrix:", confusion_matrix(y_test,svm_pred))
print("precision_score:",svm_precision)
print("recall_score:",svm_recall)
print("f1_score:",svm_f1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution
param_dist = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Initialize Random Search
random_search = RandomizedSearchCV(estimator=logistic_reg, param_distributions=param_dist, n_iter=100, cv=5, verbose=1, n_jobs=-1, random_state=42)

# Fit the model
random_search.fit(x_train, y_train)

# Get the best parameters and model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

print("Best Parameters:", best_params)
print("Best Model:", best_model)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test set
y_pred_lg_hpt = best_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_lg_hpt)
report = classification_report(y_test, y_pred_lg_hpt)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
rf_clf1=RandomForestClassifier(n_estimators=10,criterion='entropy',max_features='sqrt') # Changed max_features to 'sqrt'
rf_clf1.fit(x_train, y_train)
rf_pred1=rf_clf1.predict(x_val)

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
# Assuming you have your original training data in x_train and y_train

smote = SMOTE(random_state=42)
x_train_balanced, y_train_balanced = smote.fit_resample(x_train, y_train)

rf_clf1_b=RandomForestClassifier(n_estimators=10,criterion='gini',max_features='sqrt')
rf_clf1_b.fit(x_train_balanced, y_train_balanced)
rf_pred1_b=rf_clf1_b.predict(x_val)

In [ ]:
dt_clf1=DecisionTreeClassifier(class_weight='balanced',criterion='entropy',max_depth=200)
dt_clf1.fit(x_train, y_train)
dt_pred1=dt_clf1.predict(x_val)

In [ ]:
dt_clf1_b=DecisionTreeClassifier(class_weight='balanced',criterion='entropy')
dt_clf1_b.fit(x_train_balanced, y_train_balanced)
dt_pred1_b=dt_clf1_b.predict(x_val)

In [ ]:
mux = pd.MultiIndex.from_product([['Without Oversampling','With Oversampling'],['Random Forest','DecisionTree']])
Metric=['Train Accuracy','Validation Accuracy','Precision','Recall','F1 Score']
comp_table = pd.DataFrame([[rf_clf1.score(x_train,y_train),dt_clf1.score(x_train,y_train),
                            rf_clf1_b.score(x_train_balanced,y_train_balanced),dt_clf1_b.score(x_train_balanced,y_train_balanced)],
                            [accuracy_score(y_val,rf_pred1),accuracy_score(y_val,dt_pred1),
                            accuracy_score(y_val,rf_pred1_b),accuracy_score(y_val,dt_pred1_b)],
                            [precision_score(y_val,rf_pred1,average='weighted'),precision_score(y_val,dt_pred1,average='weighted'),
                            precision_score(y_val,rf_pred1_b,average='weighted'),precision_score(y_val,dt_pred1_b,average='weighted')],
                            [recall_score(y_val,rf_pred1,average='weighted'),recall_score(y_val,dt_pred1,average='weighted'),
                            recall_score(y_val,rf_pred1_b,average='weighted'),recall_score(y_val,dt_pred1_b,average='weighted')],
                            [f1_score(y_val,rf_pred1,average='weighted'),f1_score(y_val,dt_pred1,average='weighted'),
                            f1_score(y_val,rf_pred1_b,average='weighted'),f1_score(y_val,dt_pred1_b,average='weighted')]],columns=mux)
comp_table.index=Metric
comp_table

In [ ]:
# Import the VotingClassifier
from sklearn.ensemble import VotingClassifier

voting_clf_hard = VotingClassifier(estimators=[('a1',rf_clf1),('a2',dt_clf1)],voting='hard')
voting_clf_hard.fit(x_train, y_train)
pred = voting_clf_hard.predict(x_val)

In [ ]:
metric=['Train Accuracy','Validaion Accuracy','Precision','Recall','F1 Score']
models=['Random Forest','Decision Tree','Hard Voting Classifier']
table= pd.DataFrame([[rf_clf1.score(x_train, y_train),dt_clf1.score(x_train, y_train),voting_clf_hard.score(x_train, y_train)],[accuracy_score(y_val,rf_pred1),accuracy_score(y_val,dt_pred1),accuracy_score(y_val,pred)],[precision_score(y_val,rf_pred1,average='weighted'),precision_score(y_val,dt_pred1,average='weighted'),precision_score(y_val,pred,average='weighted')],[recall_score(y_val,rf_pred1,average='weighted'),recall_score(y_val,dt_pred1,average='weighted'),recall_score(y_val,pred,average='weighted')],[f1_score(y_val,rf_pred1,average='weighted'),f1_score(y_val,dt_pred1,average='weighted'),f1_score(y_val,pred,average='weighted')]],columns=models)
table.index=metric
table

In [ ]:
metric=['Train Accuracy','Test Accuracy','Precision','Recall','F1 Score']
table= pd.DataFrame([[dt_clf1.score(x_train,y_train)],[accuracy_score(y_test,test_pred1)],[precision_score(y_test,test_pred1,average='weighted')],[recall_score(y_test,test_pred1,average='weighted')],[f1_score(y_test,test_pred1,average='weighted')]],columns=['Decision Tree'])
table.index=metric
table

***Conclusion : Metric values got improved when we do modelling with selected features compared to modelling with full features. So use the selected features only for training and predict test data using the best model obtained - Decision Tree without oversampling.***

In [ ]:
# Assuming you want to use the Decision Tree classifier with balanced class weights
test_pred = dt_clf1_b.predict(x_test)

In [ ]:
metric=['Train Accuracy','Test Accuracy','Precision','Recall','F1 Score']
table= pd.DataFrame([[dt_clf1.score(x_train,y_train)],[accuracy_score(y_test,test_pred)],[precision_score(y_test,test_pred,average='weighted')],[recall_score(y_test,test_pred,average='weighted')],[f1_score(y_test,test_pred,average='weighted')]],columns=['Decision Tree']) #Replaced dt_clf with dt_clf1
table.index=metric
table

In [ ]:
from sklearn.model_selection import GridSearchCV

param_dist = { 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth' : [20,50,100], 'criterion' :['gini', 'entropy'] }
dtree_reg = DecisionTreeClassifier()
grid_search = GridSearchCV(dtree_reg, param_grid=param_dist)
grid_search.fit(x_train, y_train)
best_params_random = grid_search.best_params_
print(f"Best Parameters for decision tree model: {best_params_random}")

In [ ]:
dt_clf1=DecisionTreeClassifier(criterion='gini',max_depth=50,max_features='log2')
dt_clf1.fit(x_train,y_train)
test_pred1=dt_clf1.predict(x_test)

metric values are good

In [ ]:
#cross validation(using stratified k fold cross validation technique)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

strat_validator=StratifiedKFold(n_splits=10,shuffle=True)
cv_result_dt_clf=cross_val_score(dt_clf1,test_pred1.reshape(-1,1),y_test,cv=strat_validator)

In [ ]:
cv_result_dt_clf

In [ ]:
cv_result_dt_clf.mean()

Average cross validation score of Decision Tree model is 0.728.

In [ ]:
#confusion matrix
cm = confusion_matrix(y_test,test_pred1)
sns.heatmap(cm,
            annot=True,
            fmt='g',
            xticklabels=['Good','Poor','Standard'],
            yticklabels=['Good','Poor','Standard'])
plt.xlabel('Prediction',fontsize=13)
plt.ylabel('Actual',fontsize=13)
plt.show()

In [ ]:
#classification report
print(classification_report(y_test,test_pred1))

In [ ]:
#saving final model by pickling
import pickle
with open('model.pkl','wb') as model_file:
  pickle.dump(dt_clf1,model_file)